In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np

In [3]:
from tensorflow.keras.applications.xception import preprocess_input
train_path = '../../data/binary_classifier_train' # 2 levels up cos this notebook is in a subfolder

train_image_generator = ImageDataGenerator(validation_split = 0.1, horizontal_flip=True,brightness_range=(0.75,1.25), channel_shift_range=100, preprocessing_function=preprocess_input) 
# Brightness range based on this https://pillow.readthedocs.io/en/3.0.x/reference/ImageEnhance.html#PIL.ImageEnhance.Brightness


train_data_gen = train_image_generator.flow_from_directory(train_path, subset='training',class_mode='binary')
val_data_gen = train_image_generator.flow_from_directory(train_path, subset='validation',class_mode='binary')

Found 3956 images belonging to 2 classes.
Found 439 images belonging to 2 classes.


In [6]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.metrics import TopKCategoricalAccuracy
# Create Model here
model = Sequential()
model.add(Xception(include_top=False, weights='imagenet', pooling='avg'))
model.add(Dense(2,activation='softmax'))
model.layers[0].trainable = False

metric = 'binary_accuracy'
model.compile(optimizer='adam', metrics=metric, loss=['binary_crossentropy'])

83689472/83683744 [==============================] - 3s 0us/step


In [7]:
from PIL import Image
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 2048)              20861480  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 20,865,578
Trainable params: 4,098
Non-trainable params: 20,861,480
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(os.path.join('saved_models','Xception_binary_snapshot_{epoch:02d}.h5'))


In [ ]:
history = model.fit(
            train_data_gen,
            steps_per_epoch=124,
            epochs=5,
            
            validation_data=val_data_gen,
            validation_steps=14) # steps tbc
model.save('saved_models/Xception_binary.h5',save_format='h5')